# Fetch Administrative boundaries from WB ArcGIS repo based on country code

In [1]:
import requests
import geopandas as gpd
from shapely.geometry import shape

# Define the REST API URL
rest_api_url = "https://services.arcgis.com/iQ1dY19aHwbSDYIF/ArcGIS/rest/services/World_Bank_Global_Administrative_Divisions_VIEW/FeatureServer"


In [2]:
# Function to get the correct layer ID based on administrative level
def get_layer_id_for_adm(adm_level):
    layers_url = f"{rest_api_url}/layers"
    response = requests.get(layers_url, params={'f': 'json'})

    if response.status_code == 200:
        layers_info = response.json().get('layers', [])
        target_layer_name = f"WB_GAD_ADM{adm_level}"
        
        for layer in layers_info:
            if layer['name'] == target_layer_name:
                return layer['id']
        
        print(f"Layer matching {target_layer_name} not found.")
        return None
    else:
        print(f"Failed to fetch layers. Status code: {response.status_code}")
        return None

# Function to fetch the ADM data using the correct layer ID
def get_adm_data(country, adm_level):
    layer_id = get_layer_id_for_adm(adm_level)
    
    if layer_id is not None:
        query_url = f"{rest_api_url}/{layer_id}/query"
        params = {
            'where': f"ISO_A3 = '{country}'",
            'outFields': '*',
            'f': 'geojson'
        }
        
        response = requests.get(query_url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            features = data.get('features', [])
            if features:
                geometry = [shape(feature['geometry']) for feature in features]
                properties = [feature['properties'] for feature in features]
                gdf = gpd.GeoDataFrame(properties, geometry=geometry)

                return gdf
            else:
                print("No features found for the specified query.")
                return None
        else:
            print(f"Error fetching data: {response.status_code}")
            return None
    else:
        print("Invalid administrative level or layer mapping not found.")
        return None

# Mapping of administrative levels to field names
adm_field_mapping = {
    0: {'code': 'HASC_0', 'name': 'NAM_0'},
    1: {'code': 'HASC_1', 'name': 'NAM_1'},
    2: {'code': 'HASC_2', 'name': 'NAM_2'},
    # Add mappings for additional levels as needed
}


In [5]:
# Example usage
country = 'BGD'  # Tunisia's ISO_A3 code
adm_level = 2    # Administrative level (e.g., ADM1)

# Fetch the data
adm_data = get_adm_data(country, adm_level)

if adm_data is not None:
    # Get the correct field names based on the administrative level
    field_names = adm_field_mapping.get(adm_level, {})
    code_field = field_names.get('code')
    name_field = field_names.get('name')

    if code_field and name_field:
        # Extract the relevant columns
        all_adm_codes = adm_data.columns.str.contains("HASC_")
        all_adm_names = adm_data.columns.str.contains("NAM_")
        all_adm_codes = adm_data.columns[all_adm_codes].to_list()
        all_adm_names = adm_data.columns[all_adm_names].to_list()
    else:
        print(f"Field names for ADM level {adm_level} not found.")
else:
    print("Missing ADM data!")


In [6]:
print("Administrative Codes:", all_adm_codes)
print("Administrative Names:", all_adm_names)

Administrative Codes: ['HASC_0', 'HASC_1', 'HASC_2']
Administrative Names: ['NAM_0', 'NAM_1', 'NAM_1_GAUL', 'NAM_1_STAT', 'NAM_1_SRCE', 'NAM_1_NTVE', 'NAM_1_WIKI', 'NAM_2', 'NAM_2_GAUL', 'NAM_2_STAT', 'NAM_2_SRCE', 'NAM_2_NTVE', 'NAM_2_WIKI', 'NAM_0_Alt']
